<a href="https://colab.research.google.com/github/lynnseygraham/cs_assignment2/blob/main/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:

#Assign values of known variables:
e = 5
n = 35
c = 10
d = 0
M = 0

#Find two primes, p and q, that multiply to equal n:
p_q = []
for i in range(2, n):
  if n % i == 0:
    p_q.append(i)

#Find Øn:
Øn = (p_q[0]-1) * (p_q[1]-1)

#Find d such that d*e = 1 % Øn and d < Øn:
for i in range(2, Øn):
  if i * e % Øn == 1:
    d = i

#Find M by calculating c**d % n:
M = c**d % n

print('a) The value of plaintext M is:', M)
print('b) The value of the secret exponent d is:', d)

a) The value of plaintext M is: 5
b) The value of the secret exponent d is: 5


# Q2

In [ ]:
q = 11
a = 2
# Assign values of known private keys:
Y_A = 9
Y_B = 3
# Initialize variables for unknown private keys:
X_A = 0
X_B = 0

#a) What is A's private key, X_A, given that Y_A=9?
for i in range(1, 100):
  x = a**i % q
  if x == Y_A:
    X_A = i
    print('a) A\'s private key is:', X_A)
    break

#b) What is secret key K shared with A?
K = Y_B**X_A % q
print('b) The shared secret key is:', K)


a) A's private key is: 6
b) The shared secret key is: 3


#Q3


In [63]:
#a) Compute hashes of 16 sets of 3 random prime numbers less than 2^8 (256)
# Generate set of all primes < 256
prime_list = []
comp_list = []
for i in range(1, 256):
  if i > 1:
    for j in range(2, i):
      if (i % j) == 0:
        comp_list.append(i)
      else:
        prime_list.append(i)
        #print(i)
prime_set = list(set(prime_list))
comp_set = list(set(comp_list))
#print(prime_set)
#print(comp_set)

# Generate 16 sets of 3 random primes from set generated above
import random
randset = []
xor_hashes = []
print('List of 16 Valid messages:')
for i in range(0,16):
  randset.append(random.sample(prime_list, k=3))
  xor_hashes.append(randset[i][0] ^ randset[i][1] ^ randset[i][2])
  print('{:<16} {:1}'.format(str(randset[i]), ' || '+str(xor_hashes[i])))

print('\nLooking for fraud message that matches valid message\'s hash...')

comp_randset = []
fraud_hashes = []
for i in range(0, 1000):
  comp_randset.append(random.sample(comp_list, k=3))
  fraud_hashes.append(comp_randset[i][0] ^ comp_randset[i][1] ^ comp_randset[i][2])
  if fraud_hashes[i] in xor_hashes:
    index = xor_hashes.index(fraud_hashes[i])
    print('\nCollision found after', i, 'attempts:')
    print('{:<16} {:1}'.format(str(randset[index]), ' || '+str(xor_hashes[index])))
    print('{:<16} {:1}'.format(str(comp_randset[i]), ' || '+str(fraud_hashes[i])))
    break


List of 16 Valid messages:
[177, 189, 70]    || 74
[169, 221, 229]   || 145
[206, 220, 200]   || 218
[115, 125, 153]   || 151
[109, 61, 155]    || 203
[221, 153, 54]    || 114
[228, 240, 121]   || 109
[64, 252, 111]    || 211
[231, 227, 204]   || 200
[213, 146, 110]   || 41
[90, 194, 128]    || 24
[169, 122, 240]   || 35
[181, 240, 249]   || 188
[58, 108, 160]    || 246
[183, 206, 132]   || 253
[148, 208, 227]   || 167

Looking for fraud message that matches valid message's hash...

Collision found after 4 attempts:
[169, 122, 240]   || 35
[216, 216, 35]    || 35


#Q4

In [ ]:
!pip install cryptography
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes

     |████████████████████████████████| 2.6MB 2.7MB/s 


In [109]:
def main():
  for i in range(0, 10000000):
    prime_list = []
    comp_list = []
    for i in range(1, 256):
      if i > 1:
        for j in range(2, i):
          if (i % j) == 0:
            comp_list.append(i)
          else:
            prime_list.append(i)
            #print(i)
    prime_set = list(set(prime_list))
    comp_set = list(set(comp_list))
    #print(prime_set)
    #print(comp_set)

    import random
    randset = []
    sha_hashes = []
    print('List of 16 Valid messages:')
    for i in range(0,16):
      randset.append(random.sample(prime_list, k=3))
      digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
      digest.update(str(randset[i][0]).encode())
      digest.update(str(randset[i][1]).encode())
      digest.update(str(randset[i][2]).encode())
      sha_hashes.append(digest.finalize())
      print('{:<16s} {:1}'.format(str(randset[i]), ' || '+str(sha_hashes[i])))

    print('\nFirst 10/1000 hashes generated:')

    print_count = 10
    comp_randset = []
    fraud_hashes = []
    cols_found = False
    for i in range(0, 1000):
      comp_randset.append(random.sample(comp_list, k=3))
      digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
      digest.update(str(comp_randset[i][0]).encode())
      digest.update(str(comp_randset[i][1]).encode())
      digest.update(str(comp_randset[i][2]).encode())
      fraud_hashes.append(digest.finalize())

      if print_count:
          print('{:<16s} {:1}'.format(str(comp_randset[i]), ' || '+str(fraud_hashes[i])))
          print_count -= 1

      if fraud_hashes[i] in sha_hashes:
        index = sha_hashes.index(fraud_hashes[i])
        print('\nCollision found after', i, 'attempts:')
        print('{0:10} {1}'.format(str(randset[index]), ' || '+str(sha_hashes[index])))
        print('{0:10} {1}'.format(str(comp_randset[i]), ' || '+str(fraud_hashes[i])))
        cols_found = True
        break

    print('\nLooking for fraud message that matches valid message\'s hash...')

    if not cols_found:
      print('No collisions were found.')
    else:
      break

main()

Streaming output truncated to the last 5000 lines.
[200, 114, 161]   || b"N\xeb\xbaj-\xb5\xcb`\xa5\xb6\xb8\x93Ce\xe3\xcb\xf6\x96l'\x1c\x1bb\\\xde\xe6\x06^\x9c\x0cny"

Looking for fraud message that matches valid message's hash...
No collisions were found.
List of 16 Valid messages:
[158, 209, 96]    || b'\xb4f\x1e\xe3\xa5\xba\xd1\xdb\x9blB\xf6\x12\xe6)\x12*\n7\xb8\x8eU|\xa6h\xa4w\x14\x19\xf9\x12\x1b'
[102, 65, 195]    || b'\x92\xfb\xb9\xcc_\x14\xf8V\xdb\xb0h\\U\xad\xebo\x0ct\xa5\xa2\xd5D\xd3\x9fR\x07\x07\xb4%\x081!'
[253, 190, 207]   || b'v\x89\xb0;\xa1\x91\x87:c\xbc\x91N\xf5\xd4\xa7\xe5\xe3QF{\x16\xf5\xf8\xfa \xce\xf1B\xc2\x0c4c'
[196, 205, 232]   || b'\xfd`oQ\xe8\x8aU\xeb\xfe\xfc2\xd5\x9a#\x7f\x15\xb5 BpB\x7f\x17 \x00\x14+\xa3\xa2\x8a;\x91'
[145, 233, 67]    || b'\r\xc1-\xbd\xa0\x1b\x90aB\xe5s\xb7\xa9\xa0\x8c\x9e\\8\xe7\xb2\x1f\xbd\xb2\x8ek\x1a\x90z\x84\xeb\x95\x03'
[167, 201, 212]   || b'\x01\xb2q\xce\x1a\x0e\xe5|b`:\xba\xf8\x15DesV\x94\xae\xd67\xc7H\xa0\x10\xc6\x86\x8c#L\xcc'
[217,